In [3]:
import pandas as pd
import requests
import bs4
import json
import csv


In [ ]:


def get_data_gov_map():
    '''
    Get all the real esate deals in Tel-Aviv from govmap - from 1998 until now.
    50000000-50006000 include all the range of the street codes in in tel aviv (Note:Part of them empty)
    this function get json data and return datafarme with all the data 
    '''
    url =  "https://ags.govmap.gov.il/Identify/GetDealsByStreetCodes"
    payload = {
      "StreetCodes": [
        50000101
        
      ],
      "Extent": {
        "xmin": 180110.8086,
        "ymin": 666110.696799999,
        "xmax": 186930.7648,
        "ymax": 670915.339299999
      }
    }
    
    data = requests.post(url, json=payload).json()
    df = pd.DataFrame(data["data"])
    
    
    for i in range(50000000,50006000):
        payload = {
          "StreetCodes": [
            int(i)
          ],
          "Extent": {
            "xmin": 185110.8086,
            "ymin": 669110.696799999,
            "xmax": 185730.7648,
            "ymax": 669715.339299999
          }
        }
        try:
            data = requests.post(url, json=payload, timeout=10).json()
        except:
            print(f"Oops! StreetCodes:{i} is'nt working")
 
        
        if data['data'] != []:

            new_df = pd.DataFrame(data["data"])
            df = pd.concat([df,new_df],ignore_index = True)
            print(f'index: {i} , shape:{df.shape}')
            
    return df 
    

In [13]:
url =  "https://www.nadlan.gov.il/Nadlan.REST/Main/GetAssestAndDeals"

payload = {
    "MoreAssestsType": 0,
    "FillterRoomNum": 0,
    "GridDisplayType": 0,
    "ResultLable": "תל אביב -יפו",
    "ResultType": 1,
    "ObjectID": "5000",
    "ObjectIDType": "text",
    "ObjectKey": "UNIQ_ID",
    "DescLayerID": "SETL_MID_POINT",
    "Alert": None,
    "X": 180428.31832654,
    "Y": 665726.5550939,
    "Gush": "",
    "Parcel": "",
    "showLotParcel": False,
    "showLotAddress": False,
    "OriginalSearchString": "תל אביב  יפו",
    "MutipuleResults": False,
    "ResultsOptions": None,
    "CurrentLavel": 2,
    "Navs": [],
    "QueryMapParams": {
        "QueryToRun": None,
        "QueryObjectID": "5000",
        "QueryObjectType": "number",
        "QueryObjectKey": "SETL_CODE",
        "QueryDescLayerID": "KSHTANN_SETL_AREA",
        "SpacialWhereClause": None
    },
    "isHistorical": False,
    "PageNo": 3900,
    "OrderByFilled": "DEALDATETIME",
    "OrderByDescending": True,
    "Distance": 0
}
# 4000 - 2009
# 4200 - 2008
# 4500 - 2007
# 5000 - 2005 
# 5100 - 2004 
response = requests.post(url, json=payload)
json_data = response.json()
df = pd.DataFrame(json_data['AllResults'])


def get_nandlan_data(url, payload,df):
    payload['PageNo'] = 3901
    while payload['PageNo'] < 5200 :   
        print( payload['PageNo'])
        try:

            response = requests.post(url, json=payload)
            json_data = response.json()
            new_df = pd.DataFrame(json_data['AllResults'])
            if new_df.shape[0] != 0 :
                new_rows = pd.DataFrame.from_dict(json_data)
                new_df = pd.DataFrame(json_data['AllResults'])
                df = pd.concat([df,new_df],ignore_index = True)
        except:
            print(f"error:{payload['PageNo']}")
            time.sleep(30)
                    
        
        payload['PageNo'] += 1  
        
    return df
df = get_nandlan_data(url , payload ,df )
df.to_csv('Real_Estate_TLV_Nadlan.csv')

df.shape

3901
3902
3903
3904
3905
3906
3907
3908
3909
3910
3911
3912
3913
3914
3915
3916
3917
3918
3919
3920
3921
3922
3923
3924
3925
3926
3927
3928
3929
3930
3931
3932
3933
3934
3935
3936
3937
3938
3939
3940
3941
3942
3943
3944
3945
3946
3947
3948
3949
3950
3951
3952
3953
3954
3955
3956
3957
3958
3959
3960
3961
3962
3963
3964
3965
3966
3967
3968
3969
3970
3971
3972
3973
3974
3975
3976
3977
3978
3979
3980
3981
3982
3983
3984
3985
3986
3987
3988
3989
3990
3991
3992
3993
3994
3995
3996
3997
3998
3999
4000
4001
4002
4003
4004
4005
4006
4007
4008
4009
4010
4011
4012
4013
4014
4015
4016
4017
4018
4019
4020
4021
4022
4023
4024
4025
4026
4027
4028
4029
4030
4031
4032
4033
4034
4035
4036
4037
4038
4039
4040
4041
4042
4043
4044
4045
4046
4047
4048
4049
4050
4051
4052
4053
4054
4055
4056
4057
4058
4059
4060
4061
4062
4063
4064
4065
4066
4067
4068
4069
4070
4071
4072
4073
4074
4075
4076
4077
4078
4079
4080
4081
4082
4083
4084
4085
4086
4087
4088
4089
4090
4091
4092
4093
4094
4095
4096
4097
4098
4099
4100


(26000, 20)

In [2]:
# Save the data in csv file 

# gov_map_df = get_data_gov_map()
# gov_map_df.to_csv('Real_Estate_TLV_GOVMAPS.csv')

# nadlan_df = get_nandlan_data(url,payload,df)
# nadlan_df.to_csv('Real_Estate_TLV_NADLAN.csv')


(151025, 22)